# Data Preparation
This script contains all the code to process the original CSV files into a structured dataset.

In [ ]:
import csv
import subprocess
import logging
import os
import threading
from collections import Counter
from concurrent.futures import ThreadPoolExecutor
from dataclasses import dataclass, field
from typing import List, Any, TypedDict, Optional

import pandas as pd
import spacy
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import OpenAI
from pydantic import BaseModel, Field
from spacy import displacy
from spacy.matcher import PhraseMatcher, Matcher
from spacy.tokens import Span, Doc, Token
from tqdm.autonotebook import tqdm
from IPython.core.magic import register_cell_magic
from IPython import get_ipython
from word2number import w2n
import fastcoref_spacy
from collections import deque
import torch
import math


# Downloads spaCy model
subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])
# Loads spaCy
nlp = spacy.load("en_core_web_sm")

@register_cell_magic
def skip_if(line, cell):
    if eval(line):
        return
    get_ipython().run_cell(cell)

In [ ]:
# Load original datasets
news_articles_df = pd.read_csv("data/original/local_news_articles.csv")
press_releases_df = pd.read_csv("data/original/police_press_releases.csv")

# Rename news article `article_id` column to `id`
news_articles_df.rename(columns={"article_id": "id"}, inplace=True)

# Add `id` column to police press releases, continuing from the news articles ids
start = news_articles_df["id"].max() + 1
press_releases_df.insert(0, "id", range(start, start + len(press_releases_df)))

# We can save the police press releases as is; they are all valid accidents
press_releases_df.to_csv("data/police_press_releases.csv", index=False)

# 1. Replace special characters
## Why?
1. Special characters are not always supported by NLP libraries.
2. Special characters are not always converted to lowercase successfully.

In [ ]:
# Map special characters to ASCII
mapping = str.maketrans({
    "ċ": "c",
    "Ċ": "C",
    "ġ": "g",
    "Ġ": "G",
    "ħ": "h",
    "Ħ": "H",
    "ż": "z",
    "Ż": "Z",
    "“": '"',
    "”": '"',
    "‘": "'",
    "’": "'",
    "—": "-",
    "–": "-",
    "…": "...",
})

def clean_articles(df: pd.DataFrame, columns):
    """Replaces special characters in the given dataframe columns with their ASCII counterparts"""
    for col in columns:
        df[col] = df[col].astype(str).str.translate(mapping)
    return df

# Clean the two datasets
news_articles_df = clean_articles(news_articles_df, ["title", "subtitle", "content"])
press_releases_df = clean_articles(press_releases_df, ["title", "content"])

# 2.a. Remove non-related articles
Some articles in the dataset do not refer to vehicle accidents (e.g. refers to work accidents or new accident prevention policies). We need to remove these.

This is done in two ways:
1. Matching accident phrases (e.g. car crash)
2. Finding reference to a person, vehicle and accident or injury

In [ ]:
people_subj = {"man", "woman", "child", "driver", "motorist", "motorcyclist", "pedestrian"}
vehicles = {"car", "motorcycle", "truck", "van", "bus", "bicycle",
            "Audi", "BMW", "Chevrolet", "Citroen", "Dodge", "Fiat", "Ford", "Honda", "Hyundai", "Isuzu",
            "Jaguar", "Jeep", "Kia", "Kymco", "Mercedes", "Mercedes-Benz", "Mini", "Mitsubishi", "Nissan",
            "Peugeot", "Renault", "Skoda", "Subaru", "Suzuki", "Toyota", "Volkswagen", "VW", "Volvo"}
accident = {"accident", "crash", "collision"}
injuries = {"injure", "die"}

accident_phrases = [
    "car crash", "traffic accident", "road accident", "collision",
    "crashed", "crash", "hit by a car", "motorcycle accident",
    "injured in a crash", "overturned", "run over", "lost control"
]

accident_matcher = PhraseMatcher(nlp.vocab)
patterns = [nlp(text) for text in accident_phrases]
accident_matcher.add("ACCIDENT_PATTERNS", patterns)

def refers_to_accident(text: str) -> bool:
    doc = nlp(text)

    matches = accident_matcher(doc)

    # If any accident phrases are found, assume it is a valid article
    if len(matches) > 0:
        return True

    has_people = False
    has_vehicles = False
    has_accident = False
    has_injury = False

    for token in doc:
        if token.lemma_ in people_subj:
            has_people = True

        if token.lemma_ in vehicles:
            has_vehicles = True

        if token.lemma_ in accident:
            has_accident = True

        if token.lemma_ in injuries:
            has_injury = True

        # If people, vehicles and accident or injury is mentioned, assume it is a valid article
        if has_people and has_vehicles and (has_accident or has_injury):
            return True

    return False

# IDs of articles not referring to vehicle accidents
non_related_news_article_ids = []

for id, text in news_articles_df[["id", "content"]].values:
    is_accident = refers_to_accident(text)
    if not is_accident:
        non_related_news_article_ids.append(id)

os.makedirs("data/intermediate", exist_ok=True)

non_related_news_article_df = news_articles_df[news_articles_df["id"].isin(non_related_news_article_ids)]
related_news_article_df = news_articles_df[~news_articles_df["id"].isin(non_related_news_article_ids)]

# Save dataframes as CSVs to view results
non_related_news_article_df.to_csv("data/intermediate/local_news_articles_exclusions.csv", index=False)
related_news_article_df.to_csv("data/intermediate/local_news_articles.csv", index=False)

# 2.b. Using LLMs to flag non-related articles
While the previous method works quite well, some articles still get through.
To catch these, we pass the remaining articles through 3 LLMs (GPT 5 Mini, Grok 4 Fast, Deepseek R1).

The LLMs were set up through [Microsoft Foundry](https://ai.azure.com/) to have a unified API to communicate with different LLMs.

In [ ]:
"""Initialising API"""
api_version = "2025-01-01-preview"

endpoint = "https://news-analysis-resource.openai.azure.com/openai/v1/"

# Need to login using `az login --use-device-code`
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = OpenAI(
    base_url=endpoint,
    api_key=token_provider,
)

try:
    token_provider()
    run_cell = True
except:
    run_cell = False

In [ ]:
%%skip_if not run_cell

models = [("gpt-5-mini", 50), ("grok-4-fast-non-reasoning", 50), ("DeepSeek-R1-0528", 20)]

class NonAccidentIDs(BaseModel):
    ids: List[int] = Field(description="A list of ids of news articles that are not accidents")

system_prompt = """
You are a senior data scientist reviewing a semi-structured dataset of vehicle accidents news articles. The articles were obtained by simple web scraping (ex: on the tag of the article) which means that some articles do not refer to actual accidents (for example, they refer to new accident/traffic measures).

Your job is to review the given accident CSV and return a list of news article IDs that do not refer to accidents.
Be very critical! Any article which mentions a specific accident and provides details on it should not be removed.

IMPORTANT: You MUST return the data by calling the `set_non_accident_ids` function.

Do not return anything other than a function call.
"""

csv_prompt = lambda dataset_csv, start_rows, end_rows, total_rows: f"""
MAKE SURE THAT THE RETURNED IDS EXIST IN THIS CSV!

Accident CSV Data ({start_rows}-{end_rows}/{total_rows} rows):

{dataset_csv}
"""

# LLM function definition
result_function = {
    "type": "function",
    "function": {
        "name": "set_non_accident_ids",
        "description": "Set the IDs of the news articles which do not refer to an accident",
        "parameters": NonAccidentIDs.model_json_schema()
    }
}

tools = [result_function]

tqdm.set_lock(threading.RLock())

def process_model(model, num_rows_per_request, dataset_df: pd.DataFrame, system_prompt) -> set[int]:
    total_ids = set()

    with tqdm(range(0, len(dataset_df.index), num_rows_per_request), desc=f"Querying {model}", total=len(dataset_df.index), unit=" rows") as pbar:
        i = 0
        while i < len(dataset_df.index):
            try:
                # Get row range as the LLMs cannot process the entire file at once
                start = i
                end = min(i + num_rows_per_request, len(dataset_df.index))
                df_section: pd.DataFrame = dataset_df.iloc[start:end]
                df_section_csv = df_section.to_csv(index=False)

                messages = [
                    {
                        "role": "system",
                        "content": system_prompt,
                    },
                    {
                        "role": "user",
                        "content": csv_prompt(df_section_csv, start + 1, end + 1, len(dataset_df.index)),
                    }
                ]

                response = client.chat.completions.create(
                    model=model,
                    tools=tools,
                    messages=messages,
                )

                result: NonAccidentIDs = NonAccidentIDs.model_validate_json(response.choices[0].message.tool_calls[0].function.arguments)

                for id in result.ids:
                    # Throw an error if a returned ID is not in the dataset
                    if id not in dataset_df["id"].values:
                        raise ValueError(f"ID {id} not in dataset")

                total_ids.update(result.ids)

                # Update tqdm progress bar
                pbar.update(end - start)
                i += num_rows_per_request
            except Exception as e:
                # If we get an error, retry the model (i.e. do not increment i)
                logging.warning(f"Failed to query {model}: {e}")

    return total_ids

# Run LLMs in parallel
with ThreadPoolExecutor(max_workers=len(models)) as executor:
    futures = {
        executor.submit(
            process_model,
            model,
            num_rows_per_request,
            related_news_article_df,
            system_prompt
        ): model
        for model, num_rows_per_request in models
    }

    model_ids = {}

    for f in futures.keys():
        result = f.result()
        model_ids[futures[f]] = result

all_ids = list(model_ids.values())
# Combine IDs from all LLMs
ids_union = all_ids[0].union(*all_ids[1:])

# Save the LLM excluded articles as a CSV to review
related_news_article_df[related_news_article_df["id"].isin(ids_union)].to_csv("data/intermediate/local_news_articles_llm_exclusions.csv", index=False)
# Save the same LLM excluded articles as a CSV. This CSV will be manually checked and modified
related_news_article_df[related_news_article_df["id"].isin(ids_union)].to_csv("data/intermediate/local_news_articles_llm_manual_exclusions.csv", index=False)

# 2.c. Manually review LLM exclusions
The LLMs tend to flag valid articles as invalid. Since this is a short list (because we already removed the bulk of invalid articles in step 1), we can go through the list manually and remove the valid articles.

## Removed IDs
- 3699
- 1352
- 370
- 287
- 489424
- 491118
- 491371
- 494102
- 495320
- 495942
- 496362

*Note: `local_news_articles_llm_exclusions.csv` contains the excluded rows as given by the LLMs. `local_news_articles_llm_manual_exclusions.csv` contains the same excluded rows given by the LLMs, except that any rows referring to valid accidents were manually removed.*

In [ ]:
# Manual removal of valid rows
manual_ids = {3699, 1352, 370, 287, 489424, 491118, 491371, 494102, 495320, 495942, 496362}
llm_manual_exclusions = pd.read_csv("data/intermediate/local_news_articles_llm_manual_exclusions.csv")
llm_manual_exclusions = llm_manual_exclusions[~llm_manual_exclusions["id"].isin(manual_ids)]
try:
  llm_manual_exclusions.to_csv("data/intermediate/local_news_articles_llm_manual_exclusions.csv", index=False)
except:
  pass

In [ ]:
llm_manual_exclusions = pd.read_csv("data/intermediate/local_news_articles_llm_manual_exclusions.csv")

# Filtering the original news articles
news_articles_df = related_news_article_df[~related_news_article_df["id"].isin(llm_manual_exclusions["id"])]

try:
  news_articles_df.to_csv("data/local_news_articles.csv", index=False)
except:
    pass

 # Per-person data

In [ ]:
from enum import Enum


device = 'cuda' if torch.cuda.is_available() else 'cpu'
if "patched_fastcoref" not in {n for n, _ in nlp.pipeline}:
    nlp.add_pipe("patched_fastcoref", config={'model_architecture': 'LingMessCoref', 'model_path': 'biu-nlp/lingmess-coref', 'device': device})

class CheckDir(Enum):
    BACKWARD = 0
    FORWARD = 1
    ALL = 2

def remove_contained_spans(spans: list[tuple[Any, Span]]) -> list[tuple[Any, Span]]:
    filtered_spans = []
    spans = sorted(spans, key=lambda s: (s[1].end - s[1].start), reverse=True)

    for x, span in spans:
        contained = False
        for _, kept in filtered_spans:
            if span.start >= kept.start and span.end <= kept.end:
                contained = True
                break
        if not contained:
            filtered_spans.append((x, span))

    return list(sorted(filtered_spans, key=lambda s: s[1].start))

age_matcher = Matcher(nlp.vocab)

age_patterns = [
    [{"LIKE_NUM": True}, {"LOWER": "year"}, {"LOWER": "old"}],
    [{"LIKE_NUM": True}, {"IS_PUNCT": True}, {"LOWER": "year"}, {"IS_PUNCT": True, "OP": "?"}, {"LOWER": "old"}],
    [{"TEXT": ","}, {"LIKE_NUM": True}, {"IS_PUNCT": True}]
]

age_matcher.add("AGE", age_patterns)

severity_matcher = Matcher(nlp.vocab)

severity_patterns = [
    [{"LIKE_NUM": True}, {"LOWER": "year"}, {"LOWER": "old"}],
    [{"LIKE_NUM": True}, {"IS_PUNCT": True}, {"LOWER": "year"}, {"IS_PUNCT": True}, {"LOWER": "old"}],
    [{"TEXT": ","}, {"LIKE_NUM": True}, {"IS_PUNCT": True}]
]

severity_matcher.add("SEVERITY", age_patterns)

def get_ages(doc: Doc) -> list[tuple[int, Span]]:
    ages = []
    matches = age_matcher(doc)

    for match_id, start, end in matches:
        span: Span = doc[start:end]
        age_int = -1
        for token in span:
            if token.like_num:
                try:
                    age_int = int(token.text)
                except ValueError:
                    age_int = w2n.word_to_num(token.text)
                break
        if age_int <= 0 or age_int >= 120:
            continue
        ages.append((age_int, span))

    return ages

gender_words = {
    "man": "male", "male": "male", "he": "male", "him": "male", "husband": "male",
    "woman": "female", "female": "female", "she": "female", "her": "female", "wife": "female"
}

def get_genders(doc: Doc) -> list[tuple[str, Span]]:
    genders = []
    for token in doc:
        if token.lemma_ in gender_words:
            genders.append((gender_words[token.lemma_], Span(doc, token.i, token.i + 1)))
    return genders

accepted_injuries = {
    "serious": "serious",
    "slight": "minor",
    "grievous": "serious",
    "light": "minor",
    "critical": "serious",
    "fatal": "fatal",
    "bad": "serious"
 }

def get_severities(doc: Doc) -> list[tuple[str, Span]]:
    severities = []
    def adverb_to_adj(word: str):
        w = word.lower()
        if w.endswith("ously"):
            return w[:-2]
        if w.endswith("ally"):
            return w[:-2]
        if w.endswith("ly"):
            return w[:-2]
        return w

    for token in doc:
        if token.lemma_ == "injure" or token.lemma_ == "injury":
            for child in token.children:
                if child.dep_ == "advmod" or child.dep_ == "amod":
                    injury_adj = adverb_to_adj(child.lemma_)
                    if injury_adj not in accepted_injuries:
                        continue
                    severities.append((accepted_injuries[injury_adj], Span(doc, child.i, child.i + 1)))
        elif token.lemma_ in ["dead", "die", "death"]:
            severities.append(("fatal", Span(doc, token.i, token.i + 1)))

    return severities

driver_matcher = Matcher(nlp.vocab)
vehicle_dict = {
    "car": "car", 
    "bus": "bus", 
    "taxi": "car", 
    "motorcycle": "motorcycle", 
    "motorbike": "motorcycle", 
    "bike": "motorcycle", 
    "moped": "motorcycle", 
    "bicycle": "bicycle", 
    "van": "van", 
    "truck": "truck", 
    "scooter": "motorcycle", 
    "pedestrian": "pedestrian", 
    "vehicle": None
}
driver_nouns = {"motorcyclist": "motorcycle", "motorist": "car", "biker": "motorcycle"}
driver_patterns = [
    [{"LOWER": {"IN": list(vehicle_dict.keys())}, "OP": "?"}, {"LOWER": "driver"}],
    [{"LOWER": {"IN": list(driver_nouns.keys())}}]
]
driver_matcher.add("DRIVER", driver_patterns)

# Derived through ChatGPT and Gemini
vehicle_type_map = {
    "Jeep": "car",
    "Skoda Fabia": "car",
    "Toyota Tercel": "car",
    "Peugeot 307": "car",
    "Suzuki SX4": "car",
    "Mercedes": "car",
    "Toyota Hilux": "truck",
    "Ford B Max": "car",
    "Vauxhall": "car",
    "Honda Accord": "car",
    "Dacia Duster": "car",
    "Mazda Demio": "car",
    "Toyota Vitz": "car",
    "Toyota Ractis": "car",
    "BMW 525": "car",
    "BMW 1 Series": "car",
    "Peugeot 306": "car",
    "Toyota Passo": "car",
    "Nissan": "car",
    "Peugeot Partner": "van",
    "Renault Captur": "car",
    "Toyota Aygo": "car",
    "Volkswagen Polo": "car",
    "Seat Leon": "car",
    "Mercedes C180": "car",
    "Mercedes Sprinter": "van",
    "Toyota Funcargo": "car", # or van
    "Honda Civic": "car",
    "Honda Fit": "car",
    "Hyundai Getz": "car",
    "Kia Sportage": "car",
    "Toyota Auris": "car",
    "Toyota Yaris": "car",
    "Subaru Impreza": "car",
    "Volvo XC60": "car",
    "BMW i13": "car",
    "Renault Megane": "car",
    "Hyundai i10": "car",
    "Fiat Scudo": "van",
    "Maserati Levante": "car",
    "Peugeot 2008": "car",
    "BMW 318": "car",
    "BMW": "car",
    "Citroen DS3": "car",
    "Mazda 2": "car",
    "Hyundai i30": "car",
    "Suzuki Baleno": "car",
    "Toyota Aqua": "car",
    "Toyota Passo": "car",
    "Kia Picanto": "car",
    "Yamaha T115": "motorcycle",
    "Kawasaki Ninja": "motorcycle",
    "Kawazaki ZR750": "motorcycle",
    "Honda NSC110": "motorcycle",
    "Hyosung XRX": "motorcycle",
    "Renault Master": "van",
    "Suzuki Alto": "car",
    "Chevrolet Aveo": "car",
    "Toyota Vitz": "car",
    "Suzuki": "car",
    "Renault Captur": "car",
    "Ford Transit": "van",
    "Daihatsu Terios": "car",
    "Vespa": "motorcycle",
    "Surron Light Bee": "motorcycle",
    "Ford Fiesta": "car",
    "Honda CBR": "motorcycle",
    "Mercedes B180": "car",
    "Benelli bike": "motorcycle",
    "Mitsubishi L200": "truck",
    "Kymco": "motorcycle",
    "Alfa Romeo": "car",
    "Jaguar": "car",
    "Toyota Aqua": "car",
    "Honda Fit": "car",
    "Ford Transit": "van",
    "Volvo XC60": "car",
    "Kia Avella": "car",
    "Renault Trafic": "van",
    "Peugeot Tweet": "motorcycle",
    "Peugeot 508": "car",
    "Toyota Corolla": "car",
    "Jaguar XF": "car",
    "Audi Q3": "car",
    "Smart Fortwo": "car",
    **vehicle_dict
}

# Some articles refer to the vehicle type using only part of the name, so we add those as well
additional = {}
for k, v in vehicle_type_map.items():
    words = k.split()
    if len(words) > 1:
        for word in words:
            if word in additional and additional[word] != v:
                additional[word] = None
                continue
            
            # Skip short words and non-alphabetic words
            if len(word) < 3 or not any(c.isalpha() for c in word):
                continue
            
            additional[word] = v
            
vehicle_type_map.update(additional)

vehicle_type_map = {
    k.lower(): v for k, v in vehicle_type_map.items()
}

vehicle_type_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
vehicle_docs = [nlp.make_doc(name) for name in vehicle_type_map.keys()]
max_vehicle_tokens = max(len(doc) for doc in vehicle_docs)
vehicle_type_matcher.add("VEHICLES", vehicle_docs)

def get_drivers(doc: Doc) -> list[tuple[Optional[str], Span, CheckDir]]:
    def get_vehicle_from_vehicle_span(vehicle_span: Span):
        vehicle_matches = vehicle_type_matcher(vehicle_span)
        if len(vehicle_matches) > 0:
            vehicle_spans = spacy.util.filter_spans([doc[start:end] for _, start, end in vehicle_matches])
            vehicle_types = [vehicle_type_map[vs.text.lower()] for vs in vehicle_spans]
            for i, vehicle_type in enumerate(vehicle_types):
                if vehicle_type is not None:
                    return vehicle_types[i]
            return "unknown"

        return None

    drivers = []

    matches = driver_matcher(doc)

    for match_id, start, end in matches:
        span: Span = doc[start:end]

        vehicle_type = None
        for token in span:
            if token.lower_ in vehicle_dict:
                vehicle_type = vehicle_dict[token.lower_]
                break
            if token.lower_ in driver_nouns:
                vehicle_type = driver_nouns[token.lower_]
                break

        drivers.append((vehicle_type, Span(doc, start, end), CheckDir.ALL))

    for token in doc:
        if token.pos_ == "VERB" and token.lemma_.startswith(("driv", "rid")):
            if token.lemma_.startswith("rid") and Span(doc, token.i, token.i + 2).text.lower() == "riding pillion":
                continue
            span_start = min(token.i, token.head.i)
            span_end = token.i + 1
            for child in list(token.head.children) + list(token.children):
                if child.dep_ in ["compound", "nsubjpass", "dobj"]:
                    span_start = min(span_start, child.i)
                    span_end = max(span_end, child.i)

            vehicle_span = Span(doc, span_start, span_end + 1)
            vehicle_type = None
            for tok in vehicle_span:
                if tok.lower_ in vehicle_dict:
                    vehicle_type = vehicle_dict[tok.lower_]
                    break

            if vehicle_type is None:
                vehicle_type = get_vehicle_from_vehicle_span(vehicle_span)
                # If we have an invalid vehicle span
                if vehicle_type is None:
                    # Increase span size, maybe we missed part of the name
                    vehicle_span.start = max(0, vehicle_span.start - max_vehicle_tokens)
                    vehicle_span.end = min(len(doc), vehicle_span.end + max_vehicle_tokens)
                    vehicle_type = get_vehicle_from_vehicle_span(vehicle_span)
                    vehicle_type = vehicle_type if vehicle_type is not None else vehicle_span.text

            for child in token.children:
                if child.dep_ == "agent":
                    for c in child.children:
                        if c.dep_ == "pobj":
                            span_end = max(span_end, c.i)

            tense = token.morph.get("Tense")
            dir = CheckDir.ALL
            if "Past" in tense:
                dir = CheckDir.FORWARD
            elif "Pres" in tense:
                dir = CheckDir.BACKWARD

            span = Span(doc, span_start, span_end)
            drivers.append((vehicle_type, span, dir))
            # for parent in token.ancestors:
            #     if parent

    return drivers

drunk_phrases = [
    "drunk", "drunk driving", "driving under the influence", "under the influence", "intoxicated driving",
    "drink-driver", "drink driving", "intoxicated driver", "DUI", "DWI"
]
drunk_phrase_matcher = PhraseMatcher(nlp.vocab)
drunk_phrase_matcher.add("DRUNK_PHRASES", [nlp(text) for text in drunk_phrases])

drunk_patterns = [
    [{"LOWER": {"NOT_IN": ["not", "never", "no"]}}, {"LOWER": "over"}, {"LOWER": "the"}, {"LOWER": {"IN": ["alcohol", "legal"]}}, {"LOWER": "limit"}],
]
drunk_pattern_matcher = Matcher(nlp.vocab)
drunk_pattern_matcher.add("DRUNK_PATTERNS", drunk_patterns)

def get_driver_states(doc: Doc) -> list[tuple[str, Span]]:
    states = []
    matches = drunk_phrase_matcher(doc) + drunk_pattern_matcher(doc)
    for match_id, start, end in matches:
        span: Span = doc[start:end]
        states.append(("drunk", span))
    return states

person_matcher = Matcher(nlp.vocab)
person_nouns = ["man", "woman", "boy", "girl", "teen", "teenager", "baby", "person", "driver", "passenger",
    "pedestrian", "victim", "motorcyclist", "cyclist", "rider",
    "motorist", "resident", "teenager", "youth"]
vehicle_nouns = ["car", "motorcycle", "truck", "van", "bus", "bicycle"]
person_patterns = [
    [{"LOWER": {"IN": ["a", "an", "the", "one"]}, "OP": "?"}, {"LIKE_NUM": True}, {"IS_PUNCT": True}, {"LOWER": "year"}, {"IS_PUNCT": True, "OP": "?"}, {"LOWER": "old"}, {"POS": "ADJ", "OP": "?"}, {"LOWER": {"IN": person_nouns}, "OP": "?"}],
    [{"LOWER": {"IN": ["a", "an", "the", "one"]}}, {"POS": "ADJ", "OP": "?"}, {"LOWER": {"IN": person_nouns}}],
    [{"LOWER": {"IN": ["a", "an", "the", "one"]}}, {"POS": "ADJ", "OP": "?"}, {"LOWER": {"IN": vehicle_nouns}}, {"LOWER": "driver"}],
    [{"POS": "PROPN", "OP": "+"}, {"TEXT": ","}, {"LIKE_NUM": True}, {"IS_PUNCT": True}],
]

person_matcher.add("PERSON", person_patterns)

def get_people(doc: Doc) -> list[list[tuple[str, Span]]]:
    matches = person_matcher(doc)
    cluster_spans: list[list[Span]] = [
        [doc.char_span(start, end, alignment_mode="expand") for start, end in cluster] for cluster in doc._.coref_clusters
    ]

    people = []
    for match_id, start, end in matches:
        span = doc[start:end]
        people.append((span.text, span))

    cleaned = remove_contained_spans(people)

    def calc_overlap(a: Span, b: Span):
        start = max(a.start, b.start)
        end = min(a.end, b.end)

        if end <= start:
            return 0

        return end - start

    cluster_indices = []
    for text, span in cleaned:
        max_overlap = 0
        max_overlap_i = -1
        for i in range(len(cluster_spans)):
            max_cluster_overlap = max(calc_overlap(c, span) for c in cluster_spans[i])
            if max_cluster_overlap > max_overlap:
                max_overlap = max_cluster_overlap
                max_overlap_i = i

        cluster_indices.append(max_overlap_i)

    clustered_people = [
        [(c.text, c) for c in cluster]
        for i, cluster in enumerate(cluster_spans) if i in cluster_indices
    ]

    for i, person in zip(cluster_indices, cleaned):
        if i >= 0:
            continue

        clustered_people.append([person])

    return clustered_people

def get_span_ancestors(span: Span):
    def get_ancestors(token: Token):
        ancestors: dict[Token, int] = {token: 0}
        depth = 1
        current = token.head
        
        # Walk upward until we reach the root token
        while current.dep_ != "ROOT":
            ancestors[current] = depth
            depth += 1
            current = current.head
            
        ancestors[current] = depth  # Add the root token
                
        return ancestors
    
    ancestors = {}
    for tok in span:
        anc = get_ancestors(tok)
        # Combine ancestor depths, taking the minimum depth for each ancestor
        ancestors = {
            k: min(anc.get(k, float('inf')), ancestors.get(k, float('inf')))
            for k in set(anc) | set(ancestors)
        }
        
    return ancestors

def get_people_ancestors(people: list[dict[str, Span]]):
    ancestors: list[dict[str, Span]] = []
    
    for person in people:
        person_ancestors = {}
        for _, mention in person:
            mention_ancestors = get_span_ancestors(mention)
            person_ancestors = {
                k: min(mention_ancestors.get(k, float('inf')), person_ancestors.get(k, float('inf')))
                for k in set(mention_ancestors.keys()) | set(person_ancestors.keys())
            }
                
        ancestors.append(person_ancestors)
        
    return ancestors


def assign_feature_to_person(feature: Span, people: list[list[tuple[str, Span]]], people_ancestors: list[dict[Token, int]], tree_keywords: list[str] = None, check_dir: CheckDir = CheckDir.ALL) -> int:
    # Try assigning feature based on dependency tree relationships first
    feature_ancestors = get_span_ancestors(feature)
    min_depth = 10**9
    assigned_person_i = -1
    for i, anc in enumerate(people_ancestors):
        # Check if any ancestor tokens are shared between the feature and the person mentions
        common_ancestors = set(feature_ancestors.keys()).intersection(set(anc.keys()))
        if tree_keywords and len({t.lemma_ for t in common_ancestors}.intersection(tree_keywords)) > 0:
            keywords = [t for t in common_ancestors if t.lemma_ in tree_keywords]
            assigned_person = False
            for key in keywords:
                if check_dir == CheckDir.BACKWARD and feature.end < key.i:
                    continue
                elif check_dir == CheckDir.FORWARD and feature.start > key.i:
                    continue
                assigned_person_i = i
                assigned_person = True
                break
            
            if assigned_person:
                break

        person_min_depth = 10**9
        for key in common_ancestors:
            if check_dir == CheckDir.BACKWARD and feature.end < key.i:
                continue
            elif check_dir == CheckDir.FORWARD and feature.start > key.i:
                continue
            person_min_depth = min(person_min_depth, anc[key] + feature_ancestors[key])
            
        if person_min_depth < min_depth:
            min_depth = person_min_depth
            assigned_person_i = i
            
    if assigned_person_i >= 0:
        return assigned_person_i
    
    # If not tree related, fallback to distance-based assignment
    def span_distance(span_a: Span, span_b: Span, signed: bool = False):
        """
        span_a, span_b are (start, end) token index tuples
        representing half-open intervals [start, end)
        """
        a_start, a_end = span_a.start, span_a.end
        b_start, b_end = span_b.start, span_b.end

        # If spans overlap or touch, distance = 0
        if a_end >= b_start and b_end >= a_start:
            return 0

        # Otherwise distance is the gap between them
        if a_end < b_start:
            return b_start - a_end
        else:
            return -(a_start - b_end) if signed else a_start - b_end

    f_sent = feature.sent

    best_person_i = -1
    best_dist = 10**9

    for i, person in enumerate(people):
        for _, mention in person:
            if check_dir == CheckDir.BACKWARD and feature.end < mention.start:
                continue
            elif check_dir == CheckDir.FORWARD and feature.start > mention.end:
                continue
            sent_dist = min(
                abs(mention.sent.start - f_sent.start), # Same sentence
                abs(mention.sent.start - f_sent.end),   # Next sentence
                abs(f_sent.sent.start - mention.end)    # Previous sentence
            )

            # def negative_to_max(x):
            #     return x if x >= 0 else 10 ** 9

            # sent_dist = min(
            #     negative_to_max(mention.sent.start - f_sent.start),
            #     negative_to_max(f_sent.sent.start - mention.end)
            # )

            if sent_dist <= 1:
                token_dist = span_distance(mention, feature)
                if 0 <= token_dist < best_dist:
                    best_person_i = i
                    best_dist = token_dist

    return best_person_i

test_doc = nlp("The police said in a statement the accident was reported at 8.30pm and involved a Toyota Tercel driven by an 86-year-old woman from Xewkija and a Benelli bike driven by a 45-year-old from Sannat")
ppl = get_people(test_doc)
state = get_drivers(test_doc)
ppl_ancestors = get_people_ancestors(ppl)
for state_value, span, dir in state:
    idx = assign_feature_to_person(span, ppl, ppl_ancestors, check_dir=dir)
    print(f"Assigned state {state_value} to person {idx}")
displacy.render(test_doc)

@dataclass
class ValueSpan[T]:
    value: T
    span: Span

@dataclass
class PersonData:
    article_id: int
    person_id: int = -1
    # name: Optional[ValueSpan[str]] = None
    age: Optional[ValueSpan[int]] = None
    gender: Optional[ValueSpan[str]] = None
    severity: Optional[ValueSpan[str]] = field(default_factory=lambda: ValueSpan("none", None))
    is_driver: Optional[ValueSpan[bool]] = field(default_factory=lambda: ValueSpan(False, None))
    vehicle_type: Optional[ValueSpan[str]] = None
    person_state: Optional[ValueSpan[str]] = None

    def to_dict(self) -> dict:
        vars = self.__dict__.copy()
        vars.pop("article_id")
        vars.pop("person_id")
        dict_vars = {}
        for k, v in vars.items():
            dict_vars[k] = v.value if v is not None else None
            dict_vars[f"{k}_span"] = (v.span.start, v.span.end) if v is not None and v.span is not None else None
        return {
            "person_id": self.person_id,
            "article_id": self.article_id,
            **dict_vars
        }

    def combine(self, other: "PersonData") -> "PersonData":
        vars = self.__dict__.copy()
        vars.pop("article_id")
        vars.pop("person_id")
        
        default = PersonData(article_id=self.article_id)

        for var in vars.keys():
            if vars[var] != getattr(default, var):
                continue

            setattr(self, var, getattr(other, var))

        return self
    
    def is_empty(self) -> bool:
        vars = self.__dict__.copy()
        vars.pop("article_id")
        vars.pop("person_id")

        default = PersonData(article_id=self.article_id)

        for var in vars.keys():
            if vars[var] != getattr(default, var):
                return False

        return True

def compute_person_similarity(person1: PersonData, person2: PersonData) -> int:
    similarity = 0

    def is_same(a, b, default=None):
        return a is not None and b is not None and (default is None or (a.value != default and b.value != default)) and a.value == b.value

    def is_different(a, b, default=None):
        return a is not None and b is not None and (default is None or (a.value != default and b.value != default)) and a.value != b.value

    # if is_same(person1.name, person2.name):
    #     similarity += 10
    # elif is_different(person1.name, person2.name):
    #     return -1

    if is_same(person1.age, person2.age):
        similarity += 2
    elif is_different(person1.age, person2.age):
        return -1

    if is_same(person1.gender, person2.gender):
        similarity += 1
    elif is_different(person1.gender, person2.gender):
        return -1

    if is_same(person1.severity, person2.severity):
        similarity += 1
    elif is_different(person1.severity, person2.severity, "none"):
        return -1

    # No elif for is_driver as it is automatically false if no driver is mentioned
    if is_same(person1.is_driver, person2.is_driver):
        similarity += 1

    if is_same(person1.vehicle_type, person2.vehicle_type):
        similarity += 1
    elif is_different(person1.vehicle_type, person2.vehicle_type):
        return -1
    
    if is_same(person1.person_state, person2.person_state):
        similarity += 2

    return similarity

def build_person_graph(article_person_data: list[PersonData], threshold = 3):
    person_adj_graph = {i: [] for i in range(len(article_person_data))}

    for i in range(len(article_person_data) - 1):
        for j in range(i + 1, len(article_person_data)):
            sim = compute_person_similarity(article_person_data[i], article_person_data[j])
            if sim >= threshold:
                person_adj_graph[i].append(j)
                person_adj_graph[j].append(i)

    return person_adj_graph

def get_connected_people(article_person_data: list[PersonData], threshold = 3):
    person_adj_graph = build_person_graph(article_person_data, threshold)
    visited = set()
    components = []

    for node in person_adj_graph:
        if node not in visited:
            stack = [node]
            comp = []
            visited.add(node)

            while stack:
                cur = stack.pop()
                comp.append(cur)
                for n in person_adj_graph[cur]:
                    if n not in visited:
                        visited.add(n)
                        stack.append(n)

            components.append(comp)

    return components

def combine_people(article_person_data: list[PersonData], connected_persons: list[list[int]]) -> list[PersonData]:
    combined_data = []

    for connected in connected_persons:
        person = article_person_data[connected[0]]
        if len(connected) > 1:
            for idx in connected[1:]:
                person.combine(article_person_data[idx])

        combined_data.append(person)

    return combined_data

def process_article(row):
    content = row["content"]
    doc = nlp(content)

    # displacy.render(doc)
    ages = get_ages(doc)
    genders = get_genders(doc)
    severities = get_severities(doc)
    drivers = get_drivers(doc)
    driver_states = get_driver_states(doc)
    people = get_people(doc)
    people_ancestors = get_people_ancestors(people)

    article_person_data: list[PersonData] = [PersonData(article_id=row["id"]) for _ in people]

    # Check gender
    for i, person in enumerate(people):
        gender_list = []
        gender_spans = {}
        for _, mention in person:
            for token in mention:
                if token.lemma_ not in gender_words:
                    continue

                gender = gender_words[token.lemma_]
                gender_list.append(gender)
                gender_spans[gender] = Span(doc, token.i, token.i+1)

        if len(gender_list) > 0:
            count = Counter(gender_list)
            common_gender = count.most_common(1)[0][0]
            article_person_data[i].gender = ValueSpan(value=common_gender, span=gender_spans[common_gender])

    for txt, span in severities:
        person_idx = assign_feature_to_person(span, people, people_ancestors)
        if person_idx >= 0:
            article_person_data[person_idx].severity = ValueSpan(value=txt, span=span)

    for age, span in ages:
        person_idx = assign_feature_to_person(span, people, people_ancestors)
        if person_idx >= 0:
            article_person_data[person_idx].age = ValueSpan(value=age, span=span)

    for gender, span in genders:
        person_idx = assign_feature_to_person(span, people, people_ancestors)
        if person_idx >= 0 and article_person_data[person_idx].gender is None:
            # assert article_person_data[person_idx].gender is None or gender == article_person_data[person_idx].gender.value
            article_person_data[person_idx].gender = ValueSpan(value=gender, span=span)

    for vehicle_type, span, dir in drivers:
        person_idx = assign_feature_to_person(span, people, people_ancestors, tree_keywords=["drive", "driver"], check_dir=dir)
        if person_idx >= 0:
            article_person_data[person_idx].is_driver = ValueSpan(value=True, span=span)
            article_person_data[person_idx].vehicle_type = ValueSpan(value=vehicle_type, span=span)
            
    for state, span in driver_states:
        person_idx = assign_feature_to_person(span, people, people_ancestors)
        if person_idx >= 0:
            article_person_data[person_idx].person_state = ValueSpan(value=state, span=span)

    connected_people = get_connected_people(article_person_data, threshold=2)
    article_person_data = combine_people(article_person_data, connected_people)

    return article_person_data

max_workers = math.ceil(os.cpu_count() / 2) if device == 'cuda' else 1

def process_dataset(df: pd.DataFrame) -> list[PersonData]:
    person_data: list[PersonData] = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for idx, row in df.iterrows():
            futures.append(executor.submit(process_article, row))

        for f in tqdm(futures, desc="Processing articles"):
            article_person_data = f.result()
            person_data.extend(article_person_data)
            
    person_data = [p for p in person_data if not p.is_empty()]
    for i, person in enumerate(person_data):
        person.person_id = i
        
    return person_data

def save_article(data: list[PersonData], filename: str):
    person_dict_data = [p.to_dict() for p in data]
    with open(f"data/intermediate/{filename}.csv", "w", newline="") as f:
        writer = csv.DictWriter(f, person_dict_data[0].keys())
        writer.writeheader()
        writer.writerows(person_dict_data)
        
    if not os.path.exists(f"data/intermediate/{filename}_manual.csv"):
        with open(f"data/intermediate/{filename}_manual.csv", "w", newline="") as f:
            writer = csv.DictWriter(f, person_dict_data[0].keys())
            writer.writeheader()
            writer.writerows(person_dict_data)

news_article_person_data = process_dataset(news_articles_df)
press_releases_person_data = process_dataset(press_releases_df)
save_article(news_article_person_data, "local_news_articles_persons")
save_article(press_releases_person_data, "police_press_releases_persons")

# Highlighted article visualisation
To help with manual refinement of the dataset, the articles are outputted as HTML files for the following reasons:
1. Features in the article are highlighted, colour coded based on the person they refer to. This helps with quickly identifying whether features are assigned to the right person.
2. Each token shows its token index when hovered to help with manually inputting features and the corresponding token spans.
3. Additional data not in the datasets (ex: specifying which persons are referred to in the article) is displayed to provide all necessary data for efficient manual validation and refinement. 

In [ ]:

import html
import random

if "patched_fastcoref" in nlp.pipe_names:
    nlp.remove_pipe("patched_fastcoref")

angle = 0
step = 40
def random_pastel():
    global angle, step
    angle += step
    return f"hsl({angle}, 80%, 85%)"

def collect_person_spans(person: PersonData):
    spans = []

    def add(label, vs: ValueSpan):
        if vs and vs.span:
            start_char, end_char = vs.span.start_char, vs.span.end_char
            spans.append({
                "start": start_char,
                "end": end_char,
                "start_token": vs.span.start,
                "end_token": vs.span.end,
                "label": label,
                "value": vs.value
            })

    add("age", person.age)
    add("gender", person.gender)
    add("severity", person.severity)
    add("driver", person.is_driver)
    add("vehicle", person.vehicle_type)
    add("state", person.person_state)

    return spans

def tokenise_text(doc: Doc, start: int, end: int):
    span = doc.char_span(start, end, alignment_mode="expand")
    text = ""
    for token in span:
        text += f"<span class='token' title='Token {token.i}'>{html.escape(token.text)}</span>"
    return text

def highlight_article(article: str, people: list[PersonData], other_spans: list[Span] = None):
    # text = html.escape(article)
    doc = nlp(article)
    
    highlights = []
    color_map = {}
    
    for person in people:
        color_map[person.person_id] = random_pastel()
        
        tooltip = []
        for field in [
            "age", "gender", "severity",
            "is_driver", "vehicle_type", "person_state"
        ]:
            vs = getattr(person, field)
            if vs and vs.value is not None:
                tooltip.append(f"{field}: {vs.value}")
                
        tooltip_text = " | ".join(tooltip)
        
        for span in collect_person_spans(person):
            highlights.append({
                "start": span["start"],
                "end": span["end"],
                "start_token": span["start_token"],
                "end_token": span["end_token"],
                "color": color_map[person.person_id],
                "tooltip": tooltip_text,
                "person_id": person.person_id
            })
            
    highlights.sort(key=lambda x: x["start"])
        
    text = ""
    prev_end = 0
    for h in highlights:
        s, e = h["start"], h["end"]
        if prev_end >= e or prev_end >= s:
            continue
        text += tokenise_text(doc, prev_end, s) + f"<span class='hl' style='background:{h['color']}' " + f"title='Person {h['person_id']} | {h['tooltip']} | {h['start_token']} - {h['end_token']}'>" + article[s:e] + "</span>"
        prev_end = e
    text += tokenise_text(doc, prev_end, len(article))

    return f"<div class='highlighted-article'>{text}</div>"

def generate_html(articles: dict[int, str], people: list[PersonData], output_file="highlighted_articles.html"):
    rows = []

    for article_id, content in tqdm(articles.items(), desc="Generating HTML", total=len(articles)):
        article_people = [p for p in people if p.article_id == article_id]
        people_text = ""
        if len(article_people) > 0:
            if len(article_people) == 1:
                people_text = f"{article_people[0].person_id} (1 person)"
            else:
                min_id, max_id = min(p.person_id for p in article_people), max(p.person_id for p in article_people)
                people_text = f"{min_id} - {max_id} ({len(article_people)} people)"
        else:
            people_text = "No people identified"
        highlighted = highlight_article(content, article_people)

        rows.append(f"""
        <tr>
            <td>{article_id}</td>
            <td>{people_text}</td>
            <td>{highlighted}</td>
        </tr>
        """)

    html_doc = f"""
    <html>
    <head>
        <style>
            body {{ font-family: Arial; }}
            table {{ border-collapse: collapse; width: 100%; }}
            td, th {{ border: 1px solid #ccc; padding: 10px; }}
            .hl {{ border-radius: 4px; padding: 2px 4px; }}
            .token {{ white-space: pre; display: inline; vertical-align: bottom; }}
            .highlighted-article {{ display: flex; flex-wrap: wrap; gap: 0.25rem; }}
        </style>
    </head>
    <body>
        <h2>Token-Level Person Highlighting</h2>
        <table>
            <tr><th>ID</th><th>People IDs (# people)</th><th>Content</th></tr>
            {''.join(rows)}
        </table>
    </body>
    </html>
    """

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(html_doc)

    print(f"✅ HTML written to {output_file}")
    
article_dict = {row["id"]: row["content"] for idx, row in news_articles_df.iterrows()}
press_releases_dict = {row["id"]: row["content"] for idx, row in press_releases_df.iterrows()}
generate_html(article_dict, news_article_person_data, output_file="data/intermediate/highlighted_local_news_articles.html")
generate_html(press_releases_dict, press_releases_person_data, output_file="data/intermediate/highlighted_local_news_press_releases.html")